In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from vflow import Vset, init_args, dict_to_df, perturbation_stats
from functools import partial
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# fMRI Voxel Prediction

This `vflow` pipeline predicts using fMRI voxels.

In [2]:
ind = {}

def top_n_features(X, Y, n, i):
    if i not in ind:
        corr = np.abs(np.apply_along_axis(lambda x: np.corrcoef(x, Y[:, i])[0, 1], 0, X))
        ind[i] = np.argsort(corr[~np.isnan(corr)])[::-1][:n]
    return X[:, ind[i]]


def pca(X, n):
    return PCA(n_components=n, copy=True).fit(X).transform(X)


In [3]:
# load data
data_dir = "./data/fmri/"
X = np.load(data_dir + "fit_feat.npy")
Y = np.load(data_dir + "resp_dat.npy")

In [4]:
np.random.seed(14)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test),
                                             names=['X_train', 'X_test', 'y_train', 'y_test'])

# split y_train by voxel and extract top 500 correlated features per voxel
voxel_extract_funcs = [partial(lambda x, y, i: (top_n_features(x, y, 20, i), y[:, i]), i=i) for i in range(20)]
voxel_extract_set = Vset(name='voxel_extract', modules=voxel_extract_funcs, output_matching=True)
X_trains, y_trains = voxel_extract_set(X_train, y_train)
X_tests, y_tests = voxel_extract_set(X_test, y_test)

/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [5]:
# modeling
modeling_set = Vset(name='modeling', modules=[LassoCV()], module_keys=["Lasso"])
modeling_set.fit(X_trains, y_trains)

In [6]:
preds = modeling_set.predict(X_trains)

hard_metrics_set = Vset(name='hard_metrics', modules=[r2_score, explained_variance_score],
                             module_keys=["R2", "EV"])
hard_metrics = hard_metrics_set.evaluate(y_trains, preds)

df = dict_to_df(hard_metrics)
df

,init-voxel_extract,voxel_extract,init-modeling,init-modeling,init-modeling,modeling,hard_metrics,out
0,y_train,voxel_extract_0,X_train,X_train,y_train,Lasso,R2,0.203121
1,y_train,voxel_extract_1,X_train,X_train,y_train,Lasso,R2,0.275898
2,y_train,voxel_extract_2,X_train,X_train,y_train,Lasso,R2,0.224725
3,y_train,voxel_extract_3,X_train,X_train,y_train,Lasso,R2,0.202630
4,y_train,voxel_extract_4,X_train,X_train,y_train,Lasso,R2,0.167139
5,y_train,voxel_extract_5,X_train,X_train,y_train,Lasso,R2,0.228424
6,y_train,voxel_extract_6,X_train,X_train,y_train,Lasso,R2,0.247807
7,y_train,voxel_extract_7,X_train,X_train,y_train,Lasso,R2,0.264284
8,y_train,voxel_extract_8,X_train,X_train,y_train,Lasso,R2,0.178232
9,y_train,voxel_extract_9,X_train,X_train,y_train,Lasso,R2,0.055822


In [25]:
metrics_stats = perturbation_stats(df, 'voxel_extract')
metrics_stats

,voxel_extract,count,mean,std
0,voxel_extract_0,2,0.203121,7.850462e-17
1,voxel_extract_1,2,0.275898,0.000000e+00
2,voxel_extract_10,2,0.045923,7.850462e-17
3,voxel_extract_11,2,0.199126,7.850462e-17
4,voxel_extract_12,2,0.032398,0.000000e+00
5,voxel_extract_13,2,0.107258,1.570092e-16
6,voxel_extract_14,2,0.188789,0.000000e+00
7,voxel_extract_15,2,0.038167,0.000000e+00
8,voxel_extract_16,2,0.089735,0.000000e+00
9,voxel_extract_17,2,0.181334,0.000000e+00
